In [1]:
import pandas as pd
import numpy as np

In [44]:
df = pd.read_csv(open('../data/df_million.csv'))
df = df[df['text'].str.contains("https|Britain")== False]
df = df.drop('Unnamed: 0', 1)
print('有{}筆對話'.format(len(df)))

/home/tp6han/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


有1000000筆對話


In [45]:
df = df[df['text'].str.contains("00add|Britain")== False]
print('有{}筆對話'.format(len(df)))

有999368筆對話


In [46]:
df = df[df['text'].str.contains("對方已離開聊天室")== False]
df = df[df['text'].str.contains("我要配對")== False]
print('有{}筆對話'.format(len(df)))

有999314筆對話


In [70]:
df = df[df['text'].str.contains("Whosdiary 目前不支援此功能")== False]
df = df[df['text'].str.contains("Whosdiary關心您")== False]
df = df[df['text'].str.contains("Whosdiary 目前不支援貼圖功能")== False]
df = df[df['text'].str.contains("配對成功，您與陌生人連線了，")== False]
df = df[df['text'].str.contains("您與久未聯絡的好友連線了，可以開始聊天囉！")== False]
df = df[df['text'].str.contains("您與陌生人連線了，可以開始聊天囉！")== False]
df = df[df['text'].str.contains("Whosdiary 祝您聊天愉快")== False]

print('有{}筆對話'.format(len(df)))

有997023筆對話


In [87]:
df = df[df['text'].str.contains("Whosdiary 目前不支援說話功能")== False]
df = df[df['text'].str.contains("Whosdiary 目前不支援文字功能")== False]
df = df[df['text'].str.contains("Whosdiary 誰的日記」隨機聊")== False]
df = df[df['text'].str.contains("Whosdiary 目前不支援傳圖功能")== False]
df = df[df['text'].str.contains("將您與陌生人隨機配對")== False]
df = df[df['text'].str.contains("telegram.me")== False]


print('有{}筆對話'.format(len(df)))

有996996筆對話


In [118]:
df = df[df['text'].str.contains("配對中，請稍候")== False]
df = df[df['text'].str.contains("系統故障")== False]
df = df[df['text'].str.contains("請稍候")== False]
df = df[df['text'].str.contains("目前線上人數")== False]
df = df[df['text'].str.contains("尚未加入聊天室")== False]
df = df[df['text'].str.contains("已配對的聊天室")== False]
df = df[df['text'].str.contains("目前不支援")== False]


print('有{}筆對話'.format(len(df)))

有996924筆對話


In [ ]:
df = df[df['text'].str.contains("目前不支援")== False]
print('有{}筆對話'.format(len(df)))

In [113]:
df = df.reset_index(drop=True)

In [124]:
df[df['text'].str.contains("自動封鎖機制")== True]

,from,to,text,room,time
